In [1]:
import numpy as np
import pandas as pd
from skimage import io, feature, color, data
import matplotlib.pyplot as plt 
import cv2
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import time
from keras.preprocessing import image
from sklearn.metrics import accuracy_score


In [2]:
train_img_path = './data/seg_train/seg_train/'
test_img_path = './data/seg_test/seg_test/'

X_train = []
y_train = []

X_test = []
y_test = []

labels=['Buildings','Forest', 'Glacier','Mountain','Sea','Street']


In [3]:
#Extracts Histogram of Oriented Gradients of one image used for classification
def get_hog(jpeg_path):
    jpg = cv2.imread(jpeg_path)
    jpg = cv2.resize(jpg,(150,150))
    hog = feature.hog(jpg)/255.0
    return hog

In [4]:
#Fetches all images from the data folder and gets hog for each file aswell as saves the labels in different array
#Works with both train and test data
def jpeg_to_array (scene_type, img_root_path, data_type):
    scene_path = os.path.join(img_root_path,scene_type.lower())
    print('Loading ' + data_type +' images for scene type '+scene_type)
    for img in os.listdir(scene_path):
        img_path = os.path.join(scene_path,img)
        #Check if the path leads to an image
        if img_path.endswith('.jpg'):
            #Check if it is training or testing data and puts it in the right array
            if(data_type == 'Training'):
                X_train.append(get_hog(img_path))
                y_train.append(labels.index(str(scene_type)))
            if(data_type =='Testing'):
                X_test.append(get_hog(img_path))
                y_test.append(labels.index(str(scene_type)))


In [5]:
[jpeg_to_array(scene,train_img_path,'Training')for scene in labels]

Loading Training images for scene type Buildings
Loading Training images for scene type Forest
Loading Training images for scene type Glacier
Loading Training images for scene type Mountain
Loading Training images for scene type Sea
Loading Training images for scene type Street


[None, None, None, None, None, None]

In [6]:
[jpeg_to_array(scene,test_img_path,'Testing')for scene in labels]

Loading Testing images for scene type Buildings
Loading Testing images for scene type Forest
Loading Testing images for scene type Glacier
Loading Testing images for scene type Mountain
Loading Testing images for scene type Sea
Loading Testing images for scene type Street


[None, None, None, None, None, None]

In [7]:
pred = []
for i in range(11,16):
    neigh = KNeighborsClassifier(n_neighbors=i, p=1)
    neigh.fit(X_train, y_train)
    pred.append(neigh.predict(X_test))
    print('Done with n= ' + str(i))
    

Done with n= 11
Done with n= 12
Done with n= 13
Done with n= 14
Done with n= 15


In [8]:
for preds in pred:
    print(confusion_matrix(y_test,preds))
    print(accuracy_score(y_test,preds))

[[174 136  13  27  76  11]
 [  0 466   0   4   4   0]
 [  1  74 113 112 253   0]
 [  0  28  25 190 282   0]
 [  3  16   5  14 470   2]
 [ 28 290   9  11  46 117]]
0.51
[[174 140  12  33  69   9]
 [  0 466   0   4   4   0]
 [  1  81 109 107 255   0]
 [  0  27  22 199 277   0]
 [  3  16   2  17 470   2]
 [ 25 286  10  12  46 122]]
0.5133333333333333
[[168 143  13  30  71  12]
 [  0 466   0   4   4   0]
 [  1  78 104 106 264   0]
 [  0  26  18 193 288   0]
 [  3  17   2  11 475   2]
 [ 21 289   9  11  48 123]]
0.5096666666666667
[[166 146  12  29  73  11]
 [  0 466   0   3   5   0]
 [  1  82 100 108 262   0]
 [  0  27  18 194 286   0]
 [  3  18   2  13 472   2]
 [ 22 291  10  14  46 118]]
0.5053333333333333
[[165 146  12  30  73  11]
 [  0 467   0   3   4   0]
 [  1  84 102 105 261   0]
 [  0  28  18 187 292   0]
 [  3  18   3  12 472   2]
 [ 24 293   9  12  48 115]]
0.5026666666666667
